# Delete this as soon as you see it and don't know why its here.

In [3]:
import json

with open("representative_questions.json", "r") as f:
    representative_questions = json.load(f)

representative_questions["editais"]



from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vector = embeddings.embed_query("What is the capital of France?")

/tmp/ipykernel_6017/3271193723.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/home/mignoe/.conda/envs/dev/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
vector_to_edital = []
for edital in representative_questions["editais"].keys():
    for question in representative_questions["editais"][edital]:
        vector = embeddings.embed_query(question)
        vector_to_edital.append((vector, edital))




In [21]:
from scipy.spatial.distance import cosine

def find_edital_by_question(question):
    question_vector = embeddings.embed_query(question)
    closest_edital = None
    biggest_similarity = -1  # cosine similarity ranges [-1, 1]

    for vector, edital in vector_to_edital:
        cos_sim = 1 - cosine(question_vector, vector)
        if cos_sim > biggest_similarity:
            biggest_similarity = cos_sim
            closest_edital = edital

    return closest_edital


In [25]:
find_edital_by_question("Como funciona o Auxílio Emergencial para os estudantes?")

'Edital Auxílio Emergencial Estudantil (AEE) UFCG/PRAC/CGAE Nº 01/2025'

In [ ]:
from prompt import prompt 

def call_model(question):
    edital = find_edital_by_question(question)
    

